In [16]:
# ruff: noqa: N802, N803, N806, N815, N816
import os

import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

# Simple utilities for displaying generated code in the notebook
from utils import display_text

import archimedes as arc

THEME = os.environ.get("ARCHIMEDES_THEME", "dark")
arc.theme.set_theme(THEME)

# Quickstart

This section will walk through a "Hello, World!" example of C code generation and usage without getting into the details of the structure of the generated code.
The remaining sections will expand on this same basic workflow so that you can build a deeper understanding of what is generated, why, and how to use it effectively.

## Python implementation

To keep this as simple as possible, we'll work with a Python implementation of the classic Fibonnaci sequence:

In [17]:
def fib(a, b):
    return b, a + b

# Generate the first 10 Fibonacci numbers
a, b = 0, 1
for _ in range(10):
    a, b = fib(a, b)
    print(a)

1
1
2
3
5
8
13
21
34
55


## Converting to C code

Next we generate a C implementation of the Python logic using the [`codegen`](#archimedes.codegen) function, including initial values of the inputs.  Note that we have to provide names for the output variables so that the generated code can use meaningful names.

In [18]:
# Create "template" arguments for type inference
# and initialization
a, b = 0, 1

arc.codegen(fib, (a, b), return_names=("a_new", "b_new"))

This will generate several files.  For our purposes in this quick start, the only one we need to look at is `fib.h`.

In [19]:
with open("fib.h", "r") as f:
    c_code = f.read()

display_text(c_code)

```c

#ifndef FIB_H
#define FIB_H

#include "fib_kernel.h"

#ifdef __cplusplus
extern "C" {
#endif

// Input arguments struct
typedef struct {
    float a;    
    float b;    
} fib_arg_t;

// Output results struct
typedef struct {
    float a_new;
    float b_new;
} fib_res_t;

// Workspace struct
typedef struct {
    long int iw[fib_SZ_IW];
    float w[fib_SZ_W];
} fib_workspace_t;

// Runtime API
int fib_init(fib_arg_t* arg, fib_res_t* res, fib_workspace_t* workspace);
int fib_step(fib_arg_t* arg, fib_res_t* res, fib_workspace_t* workspace);


#ifdef __cplusplus
}
#endif

#endif // FIB_H
```

A few things to note about this implementation:

* **Static allocation**: The generated code uses pre-allocated workspace memory instead of dynamic memory allocation, meaning no heap usage.

* **Deterministic memory**: Requirements can be precisely calculated at compile time.

* **Fixed-Size Arrays**: All array dimensions are determined at code generation time, ensuring predictable memory usage regardless of input data.

* **No external dependencies**: The generated code is self-contained, simplifying the build process for deployment.

In brief:

| Python code | Generated C code |
|-----------------|----------------------|
| Dynamic memory allocation | Fixed, pre-allocated arrays |
| High-level array operations | Low-level pointer manipulation |
| Readable control flow | Optimized computational structure |
| Generic function for any filter order | Specialized for specific dimensions |

These details are critical for real-time and embedded applications.
However, the price for this efficiency is that the implementation hardly resembles the original Python algorithm.
This is why the automatic code generation paradigm is so helpful; the high-level algorithm can be modified in Python without the need for time-intensive re-coding the low-level C implementation.

However, at this point we still have quite a bit of work ahead of us.
In particular, we still need to:

1. Create arrays for arguments, results, and working memory
2. Map these to the pointers expected by the `iir_filter` implementation
3. Initialize the arrays correctly

If you examine the source code more carefully, you might even be surprised to see that the filter coefficients aren't even stored in the C code.
We could of course use these as global variables in Python, which would effectively "hardcode" the filter coefficients into the generated C code, but it is cleaner and more maintainable to separate the coeffients from the actual filter implementation.

This is where Archimedes builds on CasADi's powerful capabilities with its templated "driver" code generation system.
In the next part of the tutorial we'll begin to explore this by auto-generating a plain-C "main" function.